In [1]:
import pandas as pd
import os

In [2]:
import os
import logging

def get_logger(log_path):
    logger = logging.getLogger('salary_logger')
    if not logger.handlers:
        handler = logging.FileHandler(log_path)
        handler.setFormatter(logging.Formatter('%(asctime)s - %(levelname)s - %(message)s'))
        logger.addHandler(handler)
        logger.setLevel(logging.INFO)
    return logger

# Set log directory explicitly for environments where __file__ is not defined
log_directory = 'logs'
os.makedirs(log_directory, exist_ok=True)  # Create the directory if it doesn't exist
log_path = os.path.join(log_directory, 'salary_function.log')

# Use the logger
logger = get_logger(log_path)
logger.info('Logger initialized successfully.')


In [3]:
def read_data(filename):
    try:
        data = pd.read_csv(filename, low_memory=False)
        logger.info('Data loaded to dataframe sucessfully!')
        return data
        
    except FileNotFoundError:
        print(f"Error: The file {filename} was not found.")
        logger.error(f"Error: The file {filename} was not found.")
        #return f"Error: {str(e)}"
        
    except pd.errors.EmptyDataError:
        print(f"Error: The file {filename} is empty.")
        logger.error(f"Error: The file {filename} is empty.")
        #return f"Error: {str(e)}"
    
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        logger.error(f"An unexpected error occurred: {e}")
        
        

In [4]:
data = read_data("Total.csv")

In [5]:
data.head(2)

EmployeeName                                        JobTitle    BasePay  \
0  NATHANIEL FORD  GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY  167411.18   
1    GARY JIMENEZ                 CAPTAIN III (POLICE DEPARTMENT)  155966.02   

  OvertimePay   OtherPay      Benefits   TotalPay  TotalPayBenefits  Year  
0        0.00  400184.25  Not Provided  567595.43         567595.43  2011  
1   245131.88  137811.38  Not Provided  538909.28         538909.28  2011

In [6]:
def employee_details_check(name,data):    
    """
    This function get the employee details by name.
    Args:
    - name (str): Name of the employee.
    - data (pd.DataFrame): The salary data containing employee details.

    Returns:
    - details (pd.DataFrame or str): Employee details or a message if the employee is not found.
    """
    try:
        employee_details = data[data['EmployeeName'].str.lower() == name.lower()]
        #print(employee_details.info())
        if employee_details.empty: # Check if result not found
            logger.error(f"Employee {name} not found.")
            raise ValueError(f"Employee {name} not found.")            
        return employee_details
    except Exception as e:
        logger.error(f"Error: {str(e)}")
        return f"Error: {str(e)}"

In [7]:
employee_details_check("David H Vidrio",data)

EmployeeName                      JobTitle   BasePay OvertimePay  \
104400  David H Vidrio  Public Svc Aide-Public Works  13952.11      117.13   
108007  David H Vidrio             Recreation Leader   3937.41        0.00   
143258  David H Vidrio             Recreation Leader  10329.67        0.00   
186521  David H Vidrio  Public Svc Aide-Public Works      0.00        0.00   
187546  David H Vidrio             Recreation Leader    637.31        0.00   
227005  David H Vidrio             Recreation Leader   2661.31        0.00   
270456  David H Vidrio  Public Svc Aide-Public Works      0.00        0.00   
270460  David H Vidrio             Recreation Leader      0.00        0.00   

       OtherPay Benefits  TotalPay  TotalPayBenefits  Year  
104400   122.86  3165.44  14192.10          17357.54  2013  
108007     0.00   853.87   3937.41           4791.28  2013  
143258    23.59  2657.99  10353.26          13011.25  2014  
186521  2249.98   152.91   2249.98           2402.89  2015  
187546     0.00    43.32    637.31            680.63  2015  
227005     4.49  1465.23   2665.80           4131.03  2016  
270456   109.90     1.10    109.90            111.00  2017  
270460   109.90     1.10    109.90            111.00  2017

In [8]:
# Now use the new unique identifier for processing
def process_data_as_dict(data):
    """
    Process salary data into a dictionary with a unique EmployeeName,TotalBenefits,JobTitle and Year as keys.
    Args:
    - data (pd.DataFrame): The salary data containing employee details.
    
    Returns:
    - employee_dict (dict): Dictionary where unique_key are keys and employee details are values.
    """
    # Check for duplicate employee names
    duplicate_names = data['EmployeeName'][data['EmployeeName'].duplicated()]
    if len(duplicate_names) > 0:
        #print(f"Duplicate employee names found: {list(duplicate_names)}")
        print(f"Duplicate employee names found!")
        print(f"Creating a UNIQUE_KEY ")
        
        logger.info(f"Duplicate employee names found!")
        logger.info(f"Creating a UNIQUE_KEY ")
        
        data_dict = data.copy() #Make copy of data to run the dataframe to dict on. 
        # Combine EmployeeName with Year to create a unique identifier
        data_dict['unique_key'] = data_dict['EmployeeName'] + '_' + data_dict['TotalPayBenefits'].astype(str) + '_' + data_dict['JobTitle'] + '_' + data_dict['Year'].astype(str)
    else:
        print("No duplicate employee names found.")
        logger.info("No duplicate employee names found.")
        
    try:
        # Set the combined EmployeeName_Year as the index, and convert to a dictionary
        employee_dict = data_dict.set_index('unique_key').T.to_dict('list')
        logger.info("Data converted to Dict successfully!")
        return employee_dict
    except Exception as e:
        print(f"Error processing data: {e}")
        logger.error(f"Error processing data: {e}")

# Process the data into a dictionary
employee_dict = process_data_as_dict(data)

Duplicate employee names found!
Creating a UNIQUE_KEY 


In [9]:
employee_dict

{'NATHANIEL FORD_567595.43_GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY_2011': ['NATHANIEL FORD',
  'GENERAL MANAGER-METROPOLITAN TRANSIT AUTHORITY',
  '167411.18',
  '0.00',
  '400184.25',
  'Not Provided',
  567595.43,
  567595.43,
  2011],
 'GARY JIMENEZ_538909.28_CAPTAIN III (POLICE DEPARTMENT)_2011': ['GARY JIMENEZ',
  'CAPTAIN III (POLICE DEPARTMENT)',
  '155966.02',
  '245131.88',
  '137811.38',
  'Not Provided',
  538909.28,
  538909.28,
  2011],
 'ALBERT PARDINI_335279.91_CAPTAIN III (POLICE DEPARTMENT)_2011': ['ALBERT PARDINI',
  'CAPTAIN III (POLICE DEPARTMENT)',
  '212739.13',
  '106088.18',
  '16452.60',
  'Not Provided',
  335279.91,
  335279.91,
  2011],
 'CHRISTOPHER CHONG_332343.61_WIRE ROPE CABLE MAINTENANCE MECHANIC_2011': ['CHRISTOPHER CHONG',
  'WIRE ROPE CABLE MAINTENANCE MECHANIC',
  '77916.00',
  '56120.71',
  '198306.90',
  'Not Provided',
  332343.61,
  332343.61,
  2011],
 'PATRICK GARDNER_326373.19_DEPUTY CHIEF OF DEPARTMENT,(FIRE DEPARTMENT)_2011': ['PATR

In [13]:
import os
import zipfile

def export_employee_details(name, data):
    """
    Export employee details to a CSV file and save it inside a zipped folder.
    Args:
    - name (str): Name of the employee.
    - data (pd.DataFrame): The salary data containing employee details.
    
    Returns:
    - None
    """
    try:
        # Get employee details
        employee_details = employee_details_check(name, data)
        
        if isinstance(employee_details, str):  # If an error message is returned and not a dataframe type
            print(employee_details)
            logger.error(employee_details)
            return
        #print(employee_details.info())
        
        # Convert to DataFrame for exporting
        employee_df = pd.DataFrame(employee_details)#.T
        
        # Create a CSV file
        output_filename = f'{name}_details.csv'
        employee_df.to_csv(output_filename)
        logger.info(f"{output_filename} created successfully!")
        
        # Zip the CSV file
        with zipfile.ZipFile('Employee_Profile.zip', 'w') as zf:
            zf.write(output_filename)
            logger.info(f"{output_filename}.zip zipped successfully!")
        
        # Cleanup/Remove the original CSV file after zipping
        os.remove(output_filename)
        #print(f"{name}'s details exported and zipped successfully.")
        logger.info(f"{output_filename} deleted")
    
    except Exception as e:
        print(f"Error exporting employee details: {e}")
        logger.error(f"Error exporting employee details: {e}")




In [14]:
# Running checks
export_employee_details('David H Vidrio', data)